In [10]:
import sys
import os
import glob
import dlib
from rembg import remove
import cv2
import numpy as np

# Model paths
predictor_path = "shape_predictor_5_face_landmarks.dat"
face_rec_model_path = "dlib_face_recognition_resnet_model_v1.dat"

# Folder paths
faces_folder_base = "entities"  # Base path for input folders
output_folder_base = "entities_process"  # Base path for processed output folders

# Image quality assessment function
def judge_image_quality(image_path):
    """Evaluate image quality and return a score."""
    im = cv2.imread(image_path, 0)  # Read in grayscale
    blurScore = cv2.Laplacian(im, cv2.CV_64F).var()
    return str(round(blurScore))

# Load models
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

print("Starting image processing...")

# Process each subfolder
for subdir in next(os.walk(faces_folder_base))[1]:
    print(f"\nProcessing subfolder: {subdir}")
    faces_folder_path = os.path.join(faces_folder_base, subdir)
    output_folder_path = os.path.join(output_folder_base, subdir + "_process")

    # Ensure the output directory exists
    if not os.path.isdir(output_folder_path):
        os.makedirs(output_folder_path)
        print(f"Creating output directory: {output_folder_path}")

    descriptors = []
    images = []

    # Process each image
    for f in glob.glob(os.path.join(faces_folder_path, "*.jpg")):  # Supports multiple image formats
        print(f"Processing image: {f}")
        img = dlib.load_rgb_image(f)
        dets = detector(img, 1)
        print(f"Number of faces detected: {len(dets)}")

        for k, d in enumerate(dets):
            shape = sp(img, d)
            face_descriptor = facerec.compute_face_descriptor(img, shape)
            descriptors.append(face_descriptor)
            images.append((img, shape))

    # Perform clustering
    labels = dlib.chinese_whispers_clustering(descriptors, 0.5)
    num_classes = len(set(labels))
    print(f"Total number of clusters: {num_classes}")

    # Find the biggest cluster
    biggest_class_length = max([len([label for label in labels if label == i]) for i in range(num_classes)])
    biggest_class = [i for i in range(num_classes) if len([label for label in labels if label == i]) == biggest_class_length][0]
    print(f"Biggest cluster ID: {biggest_class}, contains {biggest_class_length} faces")

    # Save faces from the biggest cluster
    indices = [i for i, label in enumerate(labels) if label == biggest_class]
    for i, index in enumerate(indices):
        img, shape = images[index]
        file_path = os.path.join(output_folder_path, "face_" + str(i))
        dlib.save_face_chip(img, shape, file_path, size=450, padding=1)
        print(f"Saved face image to: {file_path}")

        # Image quality assessment and background removal
    for f in glob.glob(os.path.join(output_folder_path, "*.jpg")):
        print(f"Processing image: {f}")
        input_path = f

        # First, read the image and assess its quality
        quality_score = float(judge_image_quality(input_path))  # Ensure conversion to a floating-point number for comparison
        print(f"Image quality score: {quality_score}")

        # If image quality is less than 50, skip this image
        if quality_score < 50:
            print("Insufficient image quality, skipping.")
            os.remove(input_path)
            continue  # Skip the current iteration, do not execute the code below

        # Read the image content for background removal
        with open(input_path, 'rb') as i:
            input_image = i.read()

        # Remove the background
        output_image = remove(input_image)

        # Build a new output path, including the quality score
        output_path = input_path.replace(".jpg", f"-{quality_score}-noback.png")

        # Save the processed image to the new path
        with open(output_path, 'wb') as o:
            o.write(output_image)
            print(f"Background removal completed, saved to: {output_path}")

        # Delete the original image
        os.remove(input_path)

print("\nAll images processed.")

Starting image processing...

Processing subfolder: Justin_Trudeau
Creating output directory: entities_process/Justin_Trudeau_process
Processing image: entities/Justin_Trudeau/Justin_Trudeau_at_Eurasia_Group_2023_US-Canada_Summit_%2852807415495%29_%28cropped%29_%28cropped%29.jpg
Number of faces detected: 1
Processing image: entities/Justin_Trudeau/41796922565_54a3a07d4d_b.jpg
Number of faces detected: 0
Processing image: entities/Justin_Trudeau/24101118088_68592741cf_b.jpg
Number of faces detected: 1
Processing image: entities/Justin_Trudeau/Justin_Trudeau_March_2022.jpg
Number of faces detected: 1
Processing image: entities/Justin_Trudeau/42717355611_1019ce866c_b.jpg
Number of faces detected: 2
Processing image: entities/Justin_Trudeau/27848647717_3e4eb11fa6_b.jpg
Number of faces detected: 0
Processing image: entities/Justin_Trudeau/32795819972_2b16f408c9_b.jpg
Number of faces detected: 2
Processing image: entities/Justin_Trudeau/40907382900_681840448c_b.jpg
Number of faces detected: 

In [14]:
import os
import random
from PIL import Image, ImageFilter

# Define filter dictionary
filters = {
    "DETAIL": ImageFilter.DETAIL,
    "EDGE_ENHANCE": ImageFilter.EDGE_ENHANCE,
    "EDGE_ENHANCE_MORE": ImageFilter.EDGE_ENHANCE_MORE,
    "SHARPEN": ImageFilter.SHARPEN,
    "SMOOTH": ImageFilter.SMOOTH,
    "SMOOTH_MORE": ImageFilter.SMOOTH_MORE,
}

def apply_filter(image_path, filter_name):
    """Applies a specified filter to an image and returns the processed image object."""
    image = Image.open(image_path).convert("RGBA")
    if filter_name in filters:
        image = image.filter(filters[filter_name])
    return image

def check_overlap(new_image_rect, existing_rects):
    """Checks if the new image's position overlaps with any existing images."""
    for rect in existing_rects:
        if not (new_image_rect[2] < rect[0] or new_image_rect[0] > rect[2] or
                new_image_rect[3] < rect[1] or new_image_rect[1] > rect[3]):
            return True
    return False

def place_images_on_fixed_background(background_path, image_paths, output_path, filter_name, background_size, image_size):
    """Places multiple fixed-size images randomly on a fixed-size background image, ensuring no overlap."""
    background = Image.open(background_path).convert("RGBA")
    background = background.resize(background_size)

    images = [apply_filter(path, filter_name) for path in image_paths]
    resized_images = [img.resize(image_size) for img in images]

    placed_rects = []  # Save placed images' positions and dimensions
    for img in resized_images:
        while True:
            x = random.randint(0, background.width - img.width)
            y = random.randint(0, background.height - img.height)
            rect = (x, y, x+img.width, y+img.height)

            if not check_overlap(rect, placed_rects):
                break

        background.paste(img, (x, y), img)
        placed_rects.append(rect)

    background.save(output_path)

def get_random_image_path_from_subfolders(base_folder):
    """Randomly selects an image from each subfolder and returns a list of selected image paths."""
    image_paths = []
    for subdir in next(os.walk(base_folder))[1]:
        full_subdir_path = os.path.join(base_folder, subdir)
        files = [f for f in os.listdir(full_subdir_path) if os.path.isfile(os.path.join(full_subdir_path, f))]
        if files:
            selected_file = random.choice(files)
            image_paths.append(os.path.join(full_subdir_path, selected_file))
    return image_paths

def get_random_image_path_from_selected_folders(base_folder, selected_folders):
    """Randomly selects an image from a list of specified subfolders and returns the selected image paths."""
    image_paths = []
    for subdir in selected_folders:
        full_subdir_path = os.path.join(base_folder, subdir)
        if os.path.isdir(full_subdir_path):
            files = [f for f in os.listdir(full_subdir_path) if os.path.isfile(os.path.join(full_subdir_path, f))]
            if files:
                selected_file = random.choice(files)
                image_paths.append(os.path.join(full_subdir_path, selected_file))
    return image_paths

def generate_combined_images(selected_folders, num_images, background_path, base_output_name, filter_name, image_size):
    """Generates multiple combined images from selected folders, dynamically adjusting the canvas size based on the number of selected folders and saving them to a specified subdirectory."""
    base_folder = 'entities_process'  # Base folder path
    initial_size = (300, 200)  # Initial canvas size
    size_increment = (180, 150)  # Size increment for each additional folder

    # Calculate dynamic canvas size
    dynamic_width = initial_size[0] + size_increment[0] * (len(selected_folders) - 1)
    dynamic_height = initial_size[1] + size_increment[1] * (len(selected_folders) - 1)
    dynamic_size = (dynamic_width, dynamic_height)

    # Ensure the output subdirectory exists
    output_dir = os.path.join(base_output_name)  # Construct output directory path
    os.makedirs(output_dir, exist_ok=True)  # Create directory

    for i in range(1, num_images + 1):
        # Get image paths from selected folders
        image_paths = get_random_image_path_from_selected_folders(base_folder, selected_folders)
        # Construct each image's output path
        output_path = os.path.join(output_dir, f"{'&'.join(selected_folders)}_{i}.png")
        place_images_on_fixed_background(background_path, image_paths, output_path, filter_name, dynamic_size, image_size)

# Example usage
background_path = 'background/blue.jpg'
image_size = (150, 150)  # Size of each image to be placed
entity_names = ['Justin_Trudeau']
output_base_path = 'combined_image'  # Base path for output, now includes subdirectory
selected_filter = random.choice(list(filters.keys()))

# Function call to generate combined images
generate_combined_images(entity_names, 5, background_path, output_base_path, selected_filter, image_size)

In [15]:
import requests
import os
import env
class VerifyEmail(Exception):
    pass
class NoEmailError(Exception):
    pass
class RegistrationError(Exception):
    pass


def register(email, name, description):
    REGISTER_URL = 'https://api.openverse.org/v1/auth_tokens/register/'

    data = {'name': name,
            'description': description,
            'email': email}

    response = requests.post(REGISTER_URL, json=data)

    if response.status_code == 201:

        client_data = response.json()
        client_id = client_data['client_id']
        client_secret = client_data['client_secret']

        save_to_env(email, name, description, client_id, client_secret)
        raise VerifyEmail("check email to verify for better performance")
    else:
        print('Registration failed:', response.text)
        return None, None

def save_to_env(email, name, description, client_id, client_secret):
    with open('.env', 'w') as f:
        f.write(f'EMAIL={email}\n' if email is not None else 'EMAIL=\n')
        f.write(f'NAME={name}\n' if name is not None else 'NAME=\n')
        f.write(f'DESCRIPTION={description}\n' if description is not None else 'DESCRIPTION=\n')
        f.write(f'CLIENT_ID={client_id}\n' if client_id is not None else 'CLIENT_ID=\n')
        f.write(f'CLIENT_SECRET={client_secret}\n' if client_secret is not None else 'CLIENT_SECRET=\n')

def load_from_env():
    email = 'u6774804@anu.edu.au'
    name = 'NewsAggregator'
    description = 'This is a sample project description'
    client_id = 'SQXoubtYPuNZqo9cA4d85LYmHollCwI02lgysNu3'
    client_secret = 'ukV6KaKiQmD2sHpyGjRdbXrJKDV5SCfvWHY1hysZQG5ygSRwdzTo5GsaZZ7ihnxILgslo06iDeg3ZArme01JSJYrpmalb0VDsijVPtdYFQZmXCbfVe1CJ0jkiP1ocFez'
    return email, name, description, client_id, client_secret

email, name, description, client_id, client_secret = load_from_env()

if not (email and name and description and client_id and client_secret):
    print("Registering user...")
    email = os.getenv('EMAIL')
    if (email==None or name==None or description==None):
        save_to_env(email, name, description, client_id, client_secret)
        raise NoEmailError("no email or name or description in .env")
    client_id, client_secret = register(email, name, description)
    raise RegistrationError("no client id or secret")


def token(client_id, client_secret, grant_type="client_credentials"):
    TOKEN_URL = 'https://api.openverse.org/v1/auth_tokens/token/'

    data = {'client_id': client_id,
            'client_secret': client_secret,
            'grant_type': grant_type}


    response = requests.post(TOKEN_URL, data=data)

    if response.status_code == 200:
        return response.json()['access_token']
    else:
        raise Exception(f"Token request failed with status code {response.status_code}: {response.text}")


print("Email:", email)
print("Client ID:", client_id)
print("Client Secret:", client_secret)

access_token = token(client_id, client_secret)

print("access token:", access_token)

def key_info(access_token):
    KEY_INFO_URL = 'https://api.openverse.org/v1/rate_limit/'

    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    response = requests.get(KEY_INFO_URL, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Key info request failed with status code {response.status_code}: {response.text}")

rate_limit= key_info(access_token)

print(rate_limit)


def images_search(access_token, term):
    IMAGES_SEARCH_URL = 'https://api.openverse.org/v1/images/'

    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    params = {
        'page': 1,
        'page_size': 30,
        'q': term
    }

    response = requests.get(IMAGES_SEARCH_URL, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Image search request failed with status code {response.status_code}: {response.text}")


def download_image(image_url, folder_path):
    # Extract image file name from URL
    image_name = image_url.split('/')[-1]
    # Download the image and save it to the specified folder
    headers = {'User-Agent': "NewsAggregatorImageAgent/1.0"}

    response = requests.get(image_url, headers=headers)
    if response.status_code == 200:
        with open(os.path.join(folder_path, image_name), 'wb') as f:
            f.write(response.content)
            print(f"Image downloaded successfully: {image_name}")
    else:
        print(response)
        print(response.content)
        print(f"Failed to download image: {image_url}")


def return_image(image_url):
    # Extract image file name from URL
    image_name = image_url.split('/')[-1]
    # Download the image and save it to the specified folder
    headers = {'User-Agent': "NewsAggregatorImageAgent/1.0"}

    response = requests.get(image_url, headers=headers)
    if response.status_code == 200:
        print(f"Image downloaded successfully: {image_name}")
        return response.content
    else:
        print(response)
        print(response.content)
        print(f"Failed to download image: {image_url}")
        return None


def download_entity_image(entity):
    images=images_search(access_token, entity)
    entity_name= entity.replace(" ", "_")

    #create folder
    if not os.path.exists(os.path.join(env.faces_folder_base, entity_name)):
        os.makedirs(os.path.join(env.faces_folder_base, entity_name))

    for image in images['results']:
        title = image['title']
        download_image(image['url'], os.path.join("entities", entity_name))

download_entity_image("Justin_Trudeau")
print("download finished")


ModuleNotFoundError: No module named 'env'